In [5]:
import polars as pl
import altair as alt

In [6]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [7]:
df = pl.read_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1918
).drop_nulls(
    subset=['datum']
)

In [8]:
df.sample()

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool
"""Místopředseda PSP Karel Havlíček""","""2021ps_stenprot_074schuz_s074191.htm""",null,"""Prosím, pane místopředsedo.""",3,"""ps""",2021,74,191,2023-09-12,"""PČR, PS 2021-...""","""74. schůze""",true,true


In [15]:
import re

In [71]:
def najdi_lexy(retezec):
    return [x.replace('lex','Lex').replace('Ukrajinu','Ukrajina').replace('Ukrajině','Ukrajina').replace('Ukrajinou','Ukrajina').replace('Ukrajiny','Ukrajina') for x in re.findall(r'[Ll]ex \w{1,20}-{0,1}\w{1,20}',retezec)]

In [67]:
najdi_lexy('Přijali jsme lex Babiš, pak Lex Švancnberk, lex Pat-Mat, to bylo z lexů vše.')

['Lex Babiš', 'Lex Švancnberk', 'Lex Pat-Mat']

In [73]:
lexy = df.filter(pl.col('text').str.contains(r' [Ll]ex ')).with_columns(pl.col("text").map_elements(najdi_lexy).alias("lex")).explode('lex').group_by("lex").len().sort(by="len",descending=True)
lexy

C:\Users\micha\AppData\Local\Temp\ipykernel_26468\901567343.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  lexy = df.filter(pl.col('text').str.contains(r' [Ll]ex ')).with_columns(pl.col("text").map_elements(najdi_lexy).alias("lex")).explode('lex').group_by("lex").len().sort(by="len",descending=True)


lex,len
str,u32
"""Lex Ukrajina""",599
"""Lex Babiš""",169
"""Lex specialis""",100
"""Lex OZE""",59
"""Lex ČEZ""",32
"""Lex esto""",30
"""Lex Dukovany""",29
"""Lex Cyril""",27
"""Lex Dérer""",22


In [77]:
lexy_top = lexy.drop_nulls().filter(pl.col('len') > 10).select(pl.col('lex')).to_series().to_list()
lexy_top

['Lex Ukrajina',
 'Lex Babiš',
 'Lex specialis',
 'Lex OZE',
 'Lex ČEZ',
 'Lex esto',
 'Lex Dukovany',
 'Lex Cyril',
 'Lex Dérer',
 'Lex covid',
 'Lex Uhlíř',
 'Lex Dolanský',
 'Lex Cyrill',
 'Lex Perek',
 'Lex Meissner',
 'Lex posterior',
 'Lex imperfecta',
 'Lex Baťa',
 'Lex generalis',
 'Lex Ofner',
 'Lex Šumava',
 'Lex plyn']